# Avoiding SWAP gates

Here we provide an example that shows how we can avoid SWAP gates by only including hardware-efficient gates in our quantum circuit. We herefore study a TFIM model with nearest-neighbour coulings $J_1$ and next-nearest-neigbour coupling $J_2 \ll J_1$.

In [1]:
import matplotlib.pyplot as plt
from matplotlib import cycler
import pennylane as qml
from cqd.models import JastrowPlusSingle, LogAmplitudes
from cqd.utils import *
from cqd.tdvp import HybridTDVP
from cqd.expectation import PauliSum, PauliString
import numpy as np

plt.rcParams.update({
    "text.usetex": True, # enable latex font
    "font.family": "Helvetica", # set font style
    "text.latex.preamble": r'\usepackage{amsmath}', # add latex packages
    "font.size": "18", # set font size
    "lines.linewidth": 2, # set line width
    # "figure.dpi": 200, # set figure dpi
    "figure.figsize": [10, 6], # set figure size
})
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

linestyles = ["-", ":", "-.", "--", (0, (3, 1, 1, 1, 1, 1)), "-", "--", "-.", ":"]
markers = ["o", "v", "s", "d", "X", "p", "*", "o", "v", "s", "d", "*"]
colors = ['#c65742', '#9ad0bb', '#e4bf44', '#87584E', '#aba18d', '#332737','#c65742', '#9ad0bb', '#e4bf44', '#87584E', '#aba18d', '#332737']
colors2 = ['#003122', '#225544', '#497c6a', '#71a591', '#9ad0bb', '#c5fde7']
plt.rcParams['axes.prop_cycle'] = cycler('color', colors)

/Users/giangentinetta/Library/CloudStorage/OneDrive-epfl.ch/Documents/PhD/cqd/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Defin the Hamiltonian
def high_connectivity_ising(n_spins: int, J1: float, J2: float, h: float):
    if n_spins == 6:
        H_tilde = 0
        # add hardware efficient gates
        for i in range(n_spins - 1):
            H_tilde += J1 * qml.PauliZ(i) @ qml.PauliZ((i + 1) % n_spins)
            H_tilde += h * qml.PauliX(i)
        H_tilde += h * qml.PauliX(n_spins - 1)

        H_tilde += J1 * qml.PauliZ(0) @ qml.PauliZ(5)
        H_tilde += J1 * qml.PauliZ(1) @ qml.PauliZ(4)
        # H_tilde += J * qml.PauliZ(4) @ qml.PauliZ(7)
        # H_tilde += J * qml.PauliZ(3) @ qml.PauliZ(8)

        # Non-hardware efficient gates
        H_2 = 0
        H_2 += J2 * qml.PauliZ(0) @ qml.PauliZ(4)
        H_2 += J2 * qml.PauliZ(1) @ qml.PauliZ(3)
        H_2 += J2 * qml.PauliZ(1) @ qml.PauliZ(5)
        H_2 += J2 * qml.PauliZ(2) @ qml.PauliZ(4)

    if n_spins == 9:
        H_tilde = 0
        # add hardware efficient gates
        for i in range(n_spins - 1):
            H_tilde += J1 * qml.PauliZ(i) @ qml.PauliZ((i + 1) % n_spins)
            H_tilde += h * qml.PauliX(i)
        H_tilde += h * qml.PauliX(n_spins - 1)

        H_tilde += J1 * qml.PauliZ(0) @ qml.PauliZ(5)
        H_tilde += J1 * qml.PauliZ(1) @ qml.PauliZ(4)
        H_tilde += J1 * qml.PauliZ(3) @ qml.PauliZ(8)
        H_tilde += J1 * qml.PauliZ(4) @ qml.PauliZ(7)
        H_2 = 0
        H_2 += J2 * qml.PauliZ(0) @ qml.PauliZ(4)
        H_2 += J2 * qml.PauliZ(1) @ qml.PauliZ(3)
        H_2 += J2 * qml.PauliZ(1) @ qml.PauliZ(5)
        H_2 += J2 * qml.PauliZ(2) @ qml.PauliZ(4)
        H_2 += J2 * qml.PauliZ(3) @ qml.PauliZ(7)
        H_2 += J2 * qml.PauliZ(4) @ qml.PauliZ(6)
        H_2 += J2 * qml.PauliZ(4) @ qml.PauliZ(8)
        H_2 += J2 * qml.PauliZ(5) @ qml.PauliZ(7)

    return H_tilde, H_2

In [3]:
j1 = 0.5
j2 = 0.1
h = 1
n_spins = 9
# Create Hamiltonian
h_tilde, h_tot = high_connectivity_ising(n_spins, j1, j2, h)
h_tot = h_tot + h_tilde
h_tot = PauliSum.from_pennylane(h_tot)
h_tilde = PauliSum.from_pennylane(h_tilde)

# Create model
model = JastrowPlusSingle()  # Plug-and-play any flax module!
jax_seed = jax.random.PRNGKey(42)
theta0 = model.init(jax_seed, jnp.ones((1, n_spins)))
theta0 = zero_tree_like(theta0)
logmodel = LogAmplitudes(model)

In [4]:
# Exact simulation to benchmark
psi0 = jnp.ones(2**n_spins, dtype=complex)
psi0 /= jnp.linalg.norm(psi0)
D, V = np.linalg.eigh(h_tot.to_dense())


def exact_state(t):
    return V @ (np.diag(np.exp(-1j * D * t)) @ (V.conj().T @ psi0))

In [5]:
observables_pauli = [
    1 * PauliString(qml.Z(0) @ qml.Z(n_spins - 1), n_spins),
    1 * PauliString(qml.X(0) @ qml.X(n_spins - 1), n_spins),
]
observables_sparse = [o.to_sparse() for o in observables_pauli]

In [6]:
dt = 0.005
trotter_step = 0.1
shots = None  # Change to obtain shot-based results
trotter_order = 2
rcond = 1e-15
acond = 1e-15
trotter_correct = True
tdvp = HybridTDVP(
    h_tot,
    logmodel,
    theta0,
    h_tilde=h_tilde,
    trotter_step=trotter_step,
    shots=shots,
    trotter_order=trotter_order,
    dt=dt,
    rcond=rcond,
    acond=acond,
    correct_trotter=trotter_order,
)

zs = all_z(n_spins)

fids = []
fids_q = []
times = []
expvals = []
expvals_exact = []
expvals_q = []


def callback(t, theta, tdvp):
    state = tdvp.phi_q * logmodel(theta, zs)
    state = state / np.linalg.norm(state)
    exact = exact_state(t)
    fid = np.abs(np.vdot(exact, state)) ** 2
    fid_q = np.abs(np.vdot(tdvp.phi_q, exact)) ** 2
    print(f"t={t}, Fid={fid}, Fid_q={fid_q}", end="\r")
    fids.append(fid)
    fids_q.append(fid_q)
    times.append(t)
    evals = []
    evals_exact = []
    evals_q = []
    for obs, obs_p in zip(observables_sparse, observables_pauli):
        evals.append(tdvp.expect(obs_p, theta))
        evals_q.append(np.vdot(tdvp.phi_q, obs.dot(tdvp.phi_q)))
        evals_exact.append(np.vdot(exact, obs.dot(exact)))
    expvals.append(evals)
    expvals_exact.append(evals_exact)
    expvals_q.append(evals_q)


theta1 = tdvp.run(1, callback=callback)

KeyboardInterrupt: 

In [ ]:
# Purely classical simulation with netket
import netket.experimental as nkex
import netket as nk

ham_nk = h_tot.to_netket()
vqs = nk.vqs.FullSumState(ham_nk.hilbert, model)
vqs.parameters = zero_tree_like(vqs.parameters)
integrator = nkex.dynamics.RK45(dt)
tdvp_nk = nkex.TDVP(
    ham_nk,
    vqs,
    integrator,
)

fids_nk = []
expectation_values_nk = []
times_nk = []


def nk_callback(_, logdata, tdvp_nk: nkex.TDVP):
    state = tdvp_nk.state.to_array()
    exact = exact_state(tdvp_nk.t)
    fid = np.abs(np.vdot(exact, state)) ** 2
    fids_nk.append(fid)
    times_nk.append(tdvp_nk.t)
    evals = [tdvp_nk.state.expect(o.to_netket()).mean for o in observables_pauli]
    expectation_values_nk.append(evals)
    return True


tdvp_nk.run(1, callback=nk_callback)

In [ ]:
import matplotlib.pyplot as plt
import json

times = np.array(times)
fids = np.array(fids)
fids_q = np.array(fids_q)
times_nk = np.array(times_nk)
fids_nk = np.array(fids_nk)


trot_times = np.array([np.argmin(np.abs(times[:] - t)) for t in np.linspace(0, 1, 11)])
plt.plot(times, fids, label="CQD ansatz (statevector)")
plt.scatter(times[trot_times], fids[trot_times])
plt.plot(times, fids_q, label="Pure quantum", color=colors[2])

plt.scatter(times[trot_times], fids_q[trot_times], color=colors[2])

plt.plot(times_nk, fids_nk, label="Jastrow", color=colors[1])

plt.legend()
plt.xlabel("Time")
plt.ylabel("Fidelity")
plt.show()

In [ ]:
fig, axs = plt.subplots(2, figsize=(10, 6), sharex=True)
axs[1].set_xlabel("Time")
for i, obs_ind in enumerate([0, 1]):
    axs[i].plot(
        times,
        np.array(expvals_exact)[:, obs_ind],
        label="Exact",
        color="black",
        linestyle="--",
    )
    if obs_ind == 3:
        axs[i].legend(loc="lower left")
    axs[i].plot(
        times,
        np.array(expvals)[:, obs_ind],
        label="Trotter circuit + Jastrow",
    )
    axs[i].plot(
        times,
        np.array(expvals_q)[:, obs_ind],
        label="Hardware efficient circuit",
        color=colors[2],
    )
    axs[i].plot(
        times_nk,
        np.array(expectation_values_nk)[:, obs_ind],
        label="Jastrow",
        color=colors[1],
    )

    axs[i].scatter(
        times[trot_times],
        np.array(expvals)[trot_times, obs_ind],
        label="Trotter circuit + Jastrow",
        color=colors[0],
    )

    axs[i].scatter(
        times[trot_times],
        np.array(expvals_q)[trot_times, obs_ind],
        label="Trotter circuit",
        color=colors[2],
    )
    axs[i].set_ylabel(f"{observables_pauli[obs_ind]}")

plt.show()

Copyright 2025 Gian Gentinetta - All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.